In [1]:
# N1 : Rice dataset

In [1]:
## imports
import os
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
device = 'cpu'
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    device = 'cuda'

CUDA is available!  Training on GPU ...


In [3]:
data_transform =  transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [4]:


data_dir = '/floyd/input/rice_dataset/'
batch_size = 100
num_workers = 0


train_data = datasets.ImageFolder(
        data_dir+'training_data', 
        transform=data_transform)

test_data = datasets.ImageFolder(
        data_dir+'test_data', 
        transform=data_transform)


print(len(train_data)+len(test_data),"images")
# prepare data loaders
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
                                           num_workers=num_workers, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
                                          num_workers=num_workers, shuffle=True)



428 images


In [5]:
model = models.vgg19(pretrained=False)
#print(model)

In [6]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [7]:
import torch.nn as nn

n_classes = 4 ## Number of classes in our rice dataset

n_inputs = model.classifier[-1].in_features
last_layer = nn.Linear(n_inputs,n_classes)
model.classifier[-1] = last_layer
# if GPU is available, move the model to GPU
if train_on_gpu:
    model.cuda()
# check to see that your last layer produces the expected number of outputs
print(model.classifier)

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=4, bias=True)
)


In [8]:
import torch.optim as optim
# specify loss function (categorical cross-entropy)
criterion = nn.CrossEntropyLoss()
# specify optimizer and learning rate
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

In [9]:
## function to train the network
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train()
    train_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        
    train_loss /= len(train_loader.dataset)
    print('Epoch: {} , Training Accuracy: {}/{} ({:.0f}%) Training Loss: {:.6f}'.format(
                epoch, correct, len(train_loader.dataset),
                100. * correct / len(train_loader.dataset), train_loss))

## function to train the network
def test(model, device, criterion, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test loss: {:.4f}, Test Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
# number of epochs to train the model
n_epochs = 30

print ("Start training")

for epoch in range(1, n_epochs+1):
    
    train (model, device, train_loader, optimizer, criterion, epoch)
    test (model, device, criterion, test_loader)
    
    print ('\n')
            
print ("Done training")

Start training
Epoch: 1 , Training Accuracy: 92/348 (26%) Training Loss: 0.079457
Test loss: 0.0206, Test Accuracy: 20/80 (25%)



Epoch: 2 , Training Accuracy: 126/348 (36%) Training Loss: 0.016900
Test loss: 0.0179, Test Accuracy: 20/80 (25%)



Epoch: 3 , Training Accuracy: 128/348 (37%) Training Loss: 0.015402
Test loss: 0.0179, Test Accuracy: 20/80 (25%)



Epoch: 4 , Training Accuracy: 150/348 (43%) Training Loss: 0.014901
Test loss: 0.0176, Test Accuracy: 20/80 (25%)



Epoch: 5 , Training Accuracy: 148/348 (43%) Training Loss: 0.014841
Test loss: 0.0177, Test Accuracy: 20/80 (25%)



Epoch: 6 , Training Accuracy: 150/348 (43%) Training Loss: 0.014697
Test loss: 0.0174, Test Accuracy: 20/80 (25%)



Epoch: 7 , Training Accuracy: 151/348 (43%) Training Loss: 0.014340
Test loss: 0.0168, Test Accuracy: 20/80 (25%)



Epoch: 8 , Training Accuracy: 150/348 (43%) Training Loss: 0.014001
Test loss: 0.0158, Test Accuracy: 21/80 (26%)



Epoch: 9 , Training Accuracy: 182/348 (52%) Traini

In [11]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        if train_on_gpu:
            images, labels = images.cuda(), labels.cuda()
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the test images: 70 %


In [12]:
torch.save(model.state_dict(), 'n1.pwf')